In [4]:
import torch
import torch.nn as nn
from efficientnet_pytorch import EfficientNet

In [10]:
class BCEFocalLoss(nn.Module):
    def __init__(self, alpha=1, gamma=2, reduction='mean'):
        super(BCEFocalLoss, self).__init__()
        self.bce_loss = nn.BCEWithLogitsLoss(reduction='none')
        self.alpha = alpha
        self.gamma = gamma
        self.reduction = reduction

    def forward(self, inputs, targets):
        # Flatten the inputs and targets
        inputs = inputs.view(-1)
        targets = targets.view(-1)

        # Compute BCE loss
        bce_loss = self.bce_loss(inputs, targets)

        # Compute Focal Loss modulation factor
        pt = torch.exp(-bce_loss)  # Probability of the true class
        focal_loss = self.alpha * (1 - pt) ** self.gamma * bce_loss

        if self.reduction == 'mean':
            return focal_loss.mean()
        elif self.reduction == 'sum':
            return focal_loss.sum()
        else:
            return focal_loss

# Example usage:
# combined_loss = BCEFocalLoss(alpha=0.25, gamma=2)
# loss = combined_loss(output, target)


In [25]:
import os
import nibabel as nib

# Define the paths
images_path = "E:/kits23/2d_slices/images/"
masks_path = "E:/kits23/2d_slices/masks/"

# Function to check dimensions of images and masks
def check_image_mask_dimensions(images_path, masks_path):
    image_case_folders = sorted(os.listdir(images_path))  # Sort image case folders
    mask_case_folders = sorted(os.listdir(masks_path))    # Sort mask case folders

    # Loop through a few cases to check dimensions
    for case_folder in image_case_folders[:5]:  # Check first 5 cases, adjust as needed
        image_case_path = os.path.join(images_path, case_folder)
        mask_case_path = os.path.join(masks_path, case_folder)

        # Get image and mask files inside the case folder (they should be .nii.gz files)
        image_files = sorted([f for f in os.listdir(image_case_path) if f.endswith('.nii.gz')])
        mask_files = sorted([f for f in os.listdir(mask_case_path) if f.endswith('.nii.gz')])

        if not image_files or not mask_files:
            print(f"Warning: No valid .nii.gz files found in {image_case_path} or {mask_case_path}")
            continue

        # Load image and mask (assuming 1:1 correspondence between slices)
        image = nib.load(os.path.join(image_case_path, image_files[0])).get_fdata()
        mask = nib.load(os.path.join(mask_case_path, mask_files[0])).get_fdata()

        # Print the dimensions
        print(f"Case Folder: {case_folder}")
        print(f"Image shape: {image.shape}")
        print(f"Mask shape: {mask.shape}")
        print("-" * 50)

# Check dimensions of images and masks
check_image_mask_dimensions(images_path, masks_path)


Case Folder: 00000
Image shape: (1, 512, 512)
Mask shape: (1, 512, 512)
--------------------------------------------------
Case Folder: 00001
Image shape: (1, 512, 512)
Mask shape: (1, 512, 512)
--------------------------------------------------
Case Folder: 00002
Image shape: (1, 512, 512)
Mask shape: (1, 512, 512)
--------------------------------------------------
Case Folder: 00003
Image shape: (1, 512, 512)
Mask shape: (1, 512, 512)
--------------------------------------------------
Case Folder: 00004
Image shape: (1, 512, 512)
Mask shape: (1, 512, 512)
--------------------------------------------------


In [26]:
import os
import nibabel as nib

# Define the paths
images_path = "E:/kits23/2d_slices/images/"
masks_path = "E:/kits23/2d_slices/masks/"

# Function to check dimensions of images and masks
def check_image_mask_dimensions(images_path, masks_path):
    image_case_folders = sorted(os.listdir(images_path))  # Sort image case folders
    mask_case_folders = sorted(os.listdir(masks_path))    # Sort mask case folders

    # Loop through a few cases to check dimensions
    for case_folder in image_case_folders[:5]:  # Check first 5 cases, adjust as needed
        image_case_path = os.path.join(images_path, case_folder)
        mask_case_path = os.path.join(masks_path, case_folder)

        # Get image and mask files inside the case folder (they should be .nii.gz files)
        image_files = sorted([f for f in os.listdir(image_case_path) if f.endswith('.nii.gz')])
        mask_files = sorted([f for f in os.listdir(mask_case_path) if f.endswith('.nii.gz')])

        if not image_files or not mask_files:
            print(f"Warning: No valid .nii.gz files found in {image_case_path} or {mask_case_path}")
            continue

        # Load image and mask (assuming 1:1 correspondence between slices)
        image = nib.load(os.path.join(image_case_path, image_files[0])).get_fdata()
        mask = nib.load(os.path.join(mask_case_path, mask_files[0])).get_fdata()

        # Extract 2D slice (indexing the first dimension)
        image_slice = image[0, :, :]  # Assuming first dimension is of size 1
        mask_slice = mask[0, :, :]    # Assuming first dimension is of size 1

        # Print the dimensions
        print(f"Case Folder: {case_folder}")
        print(f"Image shape: {image_slice.shape}")
        print(f"Mask shape: {mask_slice.shape}")
        print("-" * 50)

# Check dimensions of images and masks
check_image_mask_dimensions(images_path, masks_path)


Case Folder: 00000
Image shape: (512, 512)
Mask shape: (512, 512)
--------------------------------------------------
Case Folder: 00001
Image shape: (512, 512)
Mask shape: (512, 512)
--------------------------------------------------
Case Folder: 00002
Image shape: (512, 512)
Mask shape: (512, 512)
--------------------------------------------------
Case Folder: 00003
Image shape: (512, 512)
Mask shape: (512, 512)
--------------------------------------------------
Case Folder: 00004
Image shape: (512, 512)
Mask shape: (512, 512)
--------------------------------------------------


In [1]:
import os
import random
import numpy as np
import nibabel as nib
from sklearn.model_selection import KFold

# Define paths
images_path = "E:/kits23/2d_slices/images/"
masks_path = "E:/kits23/2d_slices/masks/"

# Get all case folders
image_case_folders = sorted(os.listdir(images_path))  # List of image case folders
mask_case_folders = sorted(os.listdir(masks_path))    # List of mask case folders

# Ensure the same number of case folders in both images and masks
assert len(image_case_folders) == len(mask_case_folders), "Mismatch in number of case folders."

# Shuffle the case folders
random.seed(42)  # For reproducibility
random.shuffle(image_case_folders)

# Create KFold object for 5-fold cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Function to pad images and masks to the same number of slices
def pad_to_max_slices(image, mask, max_slices):
    # Pad image and mask to the max_slices dimension
    pad_size = max_slices - image.shape[0]
    if pad_size > 0:
        image = np.pad(image, ((0, pad_size), (0, 0), (0, 0)), mode='constant')
        mask = np.pad(mask, ((0, pad_size), (0, 0), (0, 0)), mode='constant')
    return image, mask

# Perform 5-fold cross-validation
fold = 1
for train_idx, test_idx in kf.split(image_case_folders):
    train_cases = [image_case_folders[i] for i in train_idx]
    test_cases = [image_case_folders[i] for i in test_idx]

    # Print out the train and test cases for this fold
    print(f"Fold {fold}:")
    print(f"  Training cases: {train_cases}")
    print(f"  Testing cases: {test_cases}")
    print("-" * 50)

    # Prepare the data for this fold
    train_images = []
    train_masks = []
    test_images = []
    test_masks = []

    # Collect images and masks for training
    for case_folder in train_cases:
        image_case_path = os.path.join(images_path, case_folder)
        mask_case_path = os.path.join(masks_path, case_folder)

        image_files = sorted([f for f in os.listdir(image_case_path) if f.endswith('.nii.gz')])
        mask_files = sorted([f for f in os.listdir(mask_case_path) if f.endswith('.nii.gz')])

        for image_file, mask_file in zip(image_files, mask_files):
            image = nib.load(os.path.join(image_case_path, image_file)).get_fdata()
            mask = nib.load(os.path.join(mask_case_path, mask_file)).get_fdata()

            train_images.append(image)
            train_masks.append(mask)

    # Collect images and masks for testing
    for case_folder in test_cases:
        image_case_path = os.path.join(images_path, case_folder)
        mask_case_path = os.path.join(masks_path, case_folder)

        image_files = sorted([f for f in os.listdir(image_case_path) if f.endswith('.nii.gz')])
        mask_files = sorted([f for f in os.listdir(mask_case_path) if f.endswith('.nii.gz')])

        for image_file, mask_file in zip(image_files, mask_files):
            image = nib.load(os.path.join(image_case_path, image_file)).get_fdata()
            mask = nib.load(os.path.join(mask_case_path, mask_file)).get_fdata()

            test_images.append(image)
            test_masks.append(mask)

    # Determine the maximum number of slices in the training set
    max_slices = max([img.shape[0] for img in train_images])

    # Pad images and masks for training and testing
    padded_train_images = []
    padded_train_masks = []
    for img, msk in zip(train_images, train_masks):
        padded_img, padded_msk = pad_to_max_slices(img, msk, max_slices)
        padded_train_images.append(padded_img)
        padded_train_masks.append(padded_msk)

    padded_test_images = []
    padded_test_masks = []
    for img, msk in zip(test_images, test_masks):
        padded_img, padded_msk = pad_to_max_slices(img, msk, max_slices)
        padded_test_images.append(padded_img)
        padded_test_masks.append(padded_msk)

    # Here you can now train and evaluate your model using padded_train_images and padded_train_masks
    # as the training data, and padded_test_images and padded_test_masks as the test data.

    fold += 1


Fold 1:
  Training cases: ['00129', '00041', '00045', '00230', '00094', '00198', '00484', '00555', '00004', '00175', '00437', '00458', '00487', '00188', '00532', '00209', '00007', '00242', '00021', '00050', '00096', '00196', '00288', '00293', '00059', '00284', '00132', '00144', '00552', '00505', '00184', '00267', '00213', '00164', '00412', '00528', '00170', '00139', '00089', '00422', '00266', '00526', '00424', '00074', '00037', '00286', '00088', '00240', '00297', '00127', '00414', '00160', '00092', '00100', '00558', '00109', '00404', '00182', '00469', '00120', '00093', '00179', '00026', '00568', '00413', '00199', '00095', '00155', '00419', '00223', '00253', '00241', '00444', '00085', '00415', '00483', '00481', '00038', '00227', '00069', '00106', '00439', '00143', '00565', '00531', '00031', '00177', '00008', '00261', '00461', '00141', '00580', '00211', '00068', '00448', '00072', '00251', '00002', '00442', '00207', '00076', '00244', '00407', '00178', '00268', '00560', '00538', '00066', '

KeyboardInterrupt: 

In [ ]:
import os
import random
import numpy as np
import nibabel as nib
from sklearn.model_selection import KFold

# Define paths
images_path = "E:/kits23/2d_slices/images/"
masks_path = "E:/kits23/2d_slices/masks/"

# Define output paths for saving padded images and masks
output_images_path = "E:/kits23/padded_images/"
output_masks_path = "E:/kits23/padded_masks/"

# Get all case folders
image_case_folders = sorted(os.listdir(images_path))  # List of image case folders
mask_case_folders = sorted(os.listdir(masks_path))    # List of mask case folders

# Ensure the same number of case folders in both images and masks
assert len(image_case_folders) == len(mask_case_folders), "Mismatch in number of case folders."

# Shuffle the case folders
random.seed(42)  # For reproducibility
random.shuffle(image_case_folders)

# Create KFold object for 5-fold cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Function to pad images and masks to the same number of slices
def pad_to_max_slices(image, mask, max_slices):
    # Pad image and mask to the max_slices dimension
    pad_size = max_slices - image.shape[0]
    if pad_size > 0:
        image = np.pad(image, ((0, pad_size), (0, 0), (0, 0)), mode='constant')
        mask = np.pad(mask, ((0, pad_size), (0, 0), (0, 0)), mode='constant')
    return image, mask

# Function to save padded images and masks
def save_padded_data(images, masks, output_images_path, output_masks_path, case_folder, fold_type):
    os.makedirs(output_images_path, exist_ok=True)
    os.makedirs(output_masks_path, exist_ok=True)

    for idx, (image, mask) in enumerate(zip(images, masks)):
        # Save each padded image and mask
        image_filename = f"{case_folder}_{fold_type}_slice_{idx}.nii.gz"
        mask_filename = f"{case_folder}_{fold_type}_slice_{idx}_mask.nii.gz"

        # Create NIfTI image and mask objects
        img_nifti = nib.Nifti1Image(image, np.eye(4))  # Assuming no affine transformation
        mask_nifti = nib.Nifti1Image(mask, np.eye(4))

        # Save the NIfTI files
        nib.save(img_nifti, os.path.join(output_images_path, image_filename))
        nib.save(mask_nifti, os.path.join(output_masks_path, mask_filename))

# Perform 5-fold cross-validation
fold = 1
for train_idx, test_idx in kf.split(image_case_folders):
    train_cases = [image_case_folders[i] for i in train_idx]
    test_cases = [image_case_folders[i] for i in test_idx]

    # Print out the train and test cases for this fold
    print(f"Fold {fold}:")
    print(f"  Training cases: {train_cases}")
    print(f"  Testing cases: {test_cases}")
    print("-" * 50)

    # Prepare the data for this fold
    train_images = []
    train_masks = []
    test_images = []
    test_masks = []

    # Collect images and masks for training
    for case_folder in train_cases:
        image_case_path = os.path.join(images_path, case_folder)
        mask_case_path = os.path.join(masks_path, case_folder)

        image_files = sorted([f for f in os.listdir(image_case_path) if f.endswith('.nii.gz')])
        mask_files = sorted([f for f in os.listdir(mask_case_path) if f.endswith('.nii.gz')])

        for image_file, mask_file in zip(image_files, mask_files):
            image = nib.load(os.path.join(image_case_path, image_file)).get_fdata()
            mask = nib.load(os.path.join(mask_case_path, mask_file)).get_fdata()

            train_images.append(image)
            train_masks.append(mask)

    # Collect images and masks for testing
    for case_folder in test_cases:
        image_case_path = os.path.join(images_path, case_folder)
        mask_case_path = os.path.join(masks_path, case_folder)

        image_files = sorted([f for f in os.listdir(image_case_path) if f.endswith('.nii.gz')])
        mask_files = sorted([f for f in os.listdir(mask_case_path) if f.endswith('.nii.gz')])

        for image_file, mask_file in zip(image_files, mask_files):
            image = nib.load(os.path.join(image_case_path, image_file)).get_fdata()
            mask = nib.load(os.path.join(mask_case_path, mask_file)).get_fdata()

            test_images.append(image)
            test_masks.append(mask)

    # Determine the maximum number of slices in the training set
    max_slices = max([img.shape[0] for img in train_images])

    # Pad images and masks for training and testing
    padded_train_images = []
    padded_train_masks = []
    for img, msk in zip(train_images, train_masks):
        padded_img, padded_msk = pad_to_max_slices(img, msk, max_slices)
        padded_train_images.append(padded_img)
        padded_train_masks.append(padded_msk)

    padded_test_images = []
    padded_test_masks = []
    for img, msk in zip(test_images, test_masks):
        padded_img, padded_msk = pad_to_max_slices(img, msk, max_slices)
        padded_test_images.append(padded_img)
        padded_test_masks.append(padded_msk)

    # Save padded images and masks for this fold
    save_padded_data(padded_train_images, padded_train_masks, output_images_path, output_masks_path, f"train_fold_{fold}")
    save_padded_data(padded_test_images, padded_test_masks, output_images_path, output_masks_path, f"test_fold_{fold}")

    # You can now train and evaluate your model using padded_train_images and padded_train_masks
    # for training and padded_test_images and padded_test_masks for evaluation.

    fold += 1


Fold 1:
  Training cases: ['00129', '00041', '00045', '00230', '00094', '00198', '00484', '00555', '00004', '00175', '00437', '00458', '00487', '00188', '00532', '00209', '00007', '00242', '00021', '00050', '00096', '00196', '00288', '00293', '00059', '00284', '00132', '00144', '00552', '00505', '00184', '00267', '00213', '00164', '00412', '00528', '00170', '00139', '00089', '00422', '00266', '00526', '00424', '00074', '00037', '00286', '00088', '00240', '00297', '00127', '00414', '00160', '00092', '00100', '00558', '00109', '00404', '00182', '00469', '00120', '00093', '00179', '00026', '00568', '00413', '00199', '00095', '00155', '00419', '00223', '00253', '00241', '00444', '00085', '00415', '00483', '00481', '00038', '00227', '00069', '00106', '00439', '00143', '00565', '00531', '00031', '00177', '00008', '00261', '00461', '00141', '00580', '00211', '00068', '00448', '00072', '00251', '00002', '00442', '00207', '00076', '00244', '00407', '00178', '00268', '00560', '00538', '00066', '

In [1]:
import os
import nibabel as nib

# Define the paths to the images and masks folders
image_folder = "E:/kits23/2d_slices/images/00000"
mask_folder = "E:/kits23/2d_slices/masks/00000"

# Get a list of all the files (slices) in the image and mask folders
image_files = [f for f in os.listdir(image_folder) if f.endswith('.nii.gz')]
mask_files = [f for f in os.listdir(mask_folder) if f.endswith('.nii.gz')]

# Check if the number of image files matches the number of mask files
if len(image_files) != len(mask_files):
    print(f"Warning: Number of images ({len(image_files)}) does not match number of masks ({len(mask_files)})")

# Loop through the image files and corresponding mask files
for image_file, mask_file in zip(image_files, mask_files):
    # Load the image and mask using nibabel
    image = nib.load(os.path.join(image_folder, image_file)).get_fdata()
    mask = nib.load(os.path.join(mask_folder, mask_file)).get_fdata()
    
    # Get the shape (dimensions) of the image and mask
    image_shape = image.shape
    mask_shape = mask.shape
    
    # Print the case number, image file name, and its dimensions
    case_number = os.path.basename(image_file).split('_')[0]  # Extract case number from the file name
    print(f"Case {case_number} - Image file: {image_file} - Shape: {image_shape}")
    print(f"Case {case_number} - Mask file: {mask_file} - Shape: {mask_shape}")


Case slice - Image file: slice_000.nii.gz - Shape: (1, 512, 512)
Case slice - Mask file: slice_000.nii.gz - Shape: (1, 512, 512)
Case slice - Image file: slice_001.nii.gz - Shape: (1, 512, 512)
Case slice - Mask file: slice_001.nii.gz - Shape: (1, 512, 512)
Case slice - Image file: slice_002.nii.gz - Shape: (1, 512, 512)
Case slice - Mask file: slice_002.nii.gz - Shape: (1, 512, 512)
Case slice - Image file: slice_003.nii.gz - Shape: (1, 512, 512)
Case slice - Mask file: slice_003.nii.gz - Shape: (1, 512, 512)
Case slice - Image file: slice_004.nii.gz - Shape: (1, 512, 512)
Case slice - Mask file: slice_004.nii.gz - Shape: (1, 512, 512)
Case slice - Image file: slice_005.nii.gz - Shape: (1, 512, 512)
Case slice - Mask file: slice_005.nii.gz - Shape: (1, 512, 512)
Case slice - Image file: slice_006.nii.gz - Shape: (1, 512, 512)
Case slice - Mask file: slice_006.nii.gz - Shape: (1, 512, 512)
Case slice - Image file: slice_007.nii.gz - Shape: (1, 512, 512)
Case slice - Mask file: slice_00

In [6]:
import torch
import torch.nn as nn
import torchvision.models as models
from torch.optim import Adam

# EfficientNet-B5 Encoder (from torchvision)
class EfficientNetB5Encoder(nn.Module):
    def __init__(self, pretrained=True):
        super(EfficientNetB5Encoder, self).__init__()
        # Load the pre-trained EfficientNet-B5 model from torchvision
        self.encoder = models.efficientnet_b5(pretrained=pretrained)
        
        # Remove the classification head (fully connected layers) to retain features
        self.encoder = nn.Sequential(*list(self.encoder.children())[:-1])
    
    def forward(self, x):
        # Extract features from the encoder
        features = self.encoder(x)
        return features

# FPN Decoder
class FPNDecoder(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(FPNDecoder, self).__init__()
        # FPN typically uses several layers to combine features from different scales
        self.conv1 = nn.Conv2d(in_channels, 256, kernel_size=1)
        self.conv2 = nn.Conv2d(256, 128, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(128, 64, kernel_size=3, padding=1)
        self.conv4 = nn.Conv2d(64, out_channels, kernel_size=1)
        
        # Instead of fixed upsampling, use adaptive upsampling
        self.upsample = nn.Upsample(size=(512, 512), mode='bilinear', align_corners=True)  # Adaptive resizing to (512, 512)
    
    def forward(self, x):
        # Decode and combine features from multiple scales
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        
        # Upsample the output to match the input size (512x512)
        x = self.upsample(x)
        return x

# Final model combining EfficientNet-B5 and FPN
class EfficientNetFPN(nn.Module):
    def __init__(self, out_channels=1, pretrained=True):
        super(EfficientNetFPN, self).__init__()
        # Encoder: EfficientNetB5
        self.encoder = EfficientNetB5Encoder(pretrained=pretrained)
        
        # Decoder: FPN
        self.decoder = FPNDecoder(in_channels=2048, out_channels=out_channels)  # EfficientNet-B5 has 2048 channels in the final layer

    def forward(self, x):
        # Forward pass through encoder and decoder
        features = self.encoder(x)
        out = self.decoder(features)
        return out

# Loss Function: Binary Cross-Entropy (BCE)
def bce_loss(output, target):
    return nn.BCEWithLogitsLoss()(output, target)

# Example Usage
if __name__ == "__main__":
    # Initialize the model
    model = EfficientNetFPN(out_channels=1, pretrained=True)

    # Example image size: (batch_size=2, channels=3, height=512, width=512)
    input_image = torch.randn(2, 3, 512, 512)

    # Forward pass
    output = model(input_image)
    print("Output shape: ", output.shape)  # Should output shape (batch_size, 1, 512, 512)

    # Example ground truth mask (same shape as output)
    target = torch.randn(2, 1, 512, 512)

    # Compute BCE loss
    loss = bce_loss(output, target)
    print("Loss: ", loss.item())

    # Example optimizer
    optimizer = Adam(model.parameters(), lr=0.001)

    # Backpropagation example
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()


Output shape:  torch.Size([2, 1, 512, 512])
Loss:  0.7230997681617737


In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from PIL import Image
import numpy as np
from sklearn.model_selection import KFold
from tqdm import tqdm

# Dataset class to handle image and mask loading
class KidneySegmentationDataset(Dataset):
    def __init__(self, image_dir, mask_dir, transform=None, mask_transform=None):
        self.image_dir = image_dir
        self.mask_dir = mask_dir
        self.transform = transform
        self.mask_transform = mask_transform
        
        # Collect all case folders (e.g., '00000', '00001', ...)
        self.case_folders = os.listdir(image_dir)
        self.case_folders = [folder for folder in self.case_folders if os.path.isdir(os.path.join(image_dir, folder))]
        
        # Calculate total slices across all cases and store slice counts
        self.slice_counts = []
        for case_id in self.case_folders:
            case_image_dir = os.path.join(self.image_dir, case_id)
            # Count number of images in the case
            num_slices = len([f for f in os.listdir(case_image_dir) if f.endswith('.png')])  # assuming images are .png
            self.slice_counts.append(num_slices)
        
    def __len__(self):
        # The length is the total number of slices in all case folders
        return sum(self.slice_counts)
    
    def __getitem__(self, idx):
        # Determine the case folder and slice number for the given index
        cumulative_slices = 0
        for i, num_slices in enumerate(self.slice_counts):
            cumulative_slices += num_slices
            if idx < cumulative_slices:
                case_id = self.case_folders[i]
                slice_id = idx - (cumulative_slices - num_slices)
                break  # Slice indexing
        
        # Build the paths for the image and mask slice
        img_files = sorted([f for f in os.listdir(os.path.join(self.image_dir, case_id)) if f.endswith('.png')])  # Assuming PNG files
        mask_files = sorted([f for f in os.listdir(os.path.join(self.mask_dir, case_id)) if f.endswith('.png')])  # Assuming PNG files
        
        img_path = os.path.join(self.image_dir, case_id, img_files[slice_id])
        mask_path = os.path.join(self.mask_dir, case_id, mask_files[slice_id])

        # Load the image and mask
        img = Image.open(img_path).convert("RGB")  # Convert to RGB (3 channels)
        mask = Image.open(mask_path).convert("L")  # Convert to grayscale (1 channel)
        
        # Apply any transformations if provided
        if self.transform:
            img = self.transform(img)
        if self.mask_transform:
            mask = self.mask_transform(mask)  # The mask transformation will not change the number of channels
        
        return img, mask

# EfficientNet-B5 Encoder (from torchvision)
class EfficientNetB5Encoder(nn.Module):
    def __init__(self, pretrained=True):
        super(EfficientNetB5Encoder, self).__init__()
        # Load the pre-trained EfficientNet-B5 model from torchvision
        self.encoder = models.efficientnet_b5(pretrained=pretrained)
        
        # Remove the classification head (fully connected layers) to retain features
        self.encoder = nn.Sequential(*list(self.encoder.children())[:-1])
    
    def forward(self, x):
        # Extract features from the encoder
        features = self.encoder(x)
        return features

# FPN Decoder
class FPNDecoder(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(FPNDecoder, self).__init__()
        # FPN typically uses several layers to combine features from different scales
        self.conv1 = nn.Conv2d(in_channels, 256, kernel_size=1)
        self.conv2 = nn.Conv2d(256, 128, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(128, 64, kernel_size=3, padding=1)
        self.conv4 = nn.Conv2d(64, out_channels, kernel_size=1)
        
        # Instead of fixed upsampling, use adaptive upsampling
        self.upsample = nn.Upsample(size=(512, 512), mode='bilinear', align_corners=True)  # Adaptive resizing to (512, 512)
    
    def forward(self, x):
        # Decode and combine features from multiple scales
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        
        # Upsample the output to match the input size (512x512)
        x = self.upsample(x)
        return x

# Final model combining EfficientNet-B5 and FPN
class EfficientNetFPN(nn.Module):
    def __init__(self, out_channels=1, pretrained=True):
        super(EfficientNetFPN, self).__init__()
        # Encoder: EfficientNetB5
        self.encoder = EfficientNetB5Encoder(pretrained=pretrained)
        
        # Decoder: FPN
        self.decoder = FPNDecoder(in_channels=2048, out_channels=out_channels)  # EfficientNet-B5 has 2048 channels in the final layer

    def forward(self, x):
        # Forward pass through encoder and decoder
        features = self.encoder(x)
        out = self.decoder(features)
        return out

# Loss Function: Binary Cross-Entropy (BCE)
def bce_loss(output, target):
    return nn.BCEWithLogitsLoss()(output, target)

# Training loop with 5-fold cross-validation
def train_model(image_dir, mask_dir, epochs=50, batch_size=2, lr=0.001, num_folds=5,checkpoint_dir='checkpoints'):
    # Define transformations
    transform = transforms.Compose([  # Transformation for the image
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Example mean/std for RGB images
    ])
    
    mask_transform = transforms.Compose([  # Transformation for the mask
        transforms.ToTensor(),
    ])
    
    # Create checkpoint directory if it doesn't exist
    if not os.path.exists(checkpoint_dir):
        os.makedirs(checkpoint_dir)
    
    # Initialize the dataset
    dataset = KidneySegmentationDataset(image_dir=image_dir, mask_dir=mask_dir, transform=transform, mask_transform=mask_transform)
    
    # Set up cross-validation
    kfold = KFold(n_splits=num_folds, shuffle=True)
    
    fold = 1
    for train_idx, val_idx in kfold.split(dataset):
        print(f"Training fold {fold}/{num_folds}...")
        
        # Split the dataset into training and validation sets for this fold
        train_subset = torch.utils.data.Subset(dataset, train_idx)
        val_subset = torch.utils.data.Subset(dataset, val_idx)
        
        # Data loaders
        train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=True)
        val_loader = DataLoader(val_subset, batch_size=batch_size, shuffle=False)
        
        # Initialize model, optimizer, and loss function
        model = EfficientNetFPN(out_channels=1, pretrained=True)
        
        # Check if CUDA is available, else use CPU
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        model = model.to(device)
        
        optimizer = optim.Adam(model.parameters(), lr=lr)
        
        # Load checkpoint if it exists
        checkpoint_path = os.path.join(checkpoint_dir, f"fold_{fold}_checkpoint.pth")
        if os.path.exists(checkpoint_path):
            checkpoint = torch.load(checkpoint_path)
            model.load_state_dict(checkpoint['model_state_dict'])
            optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
            start_epoch = checkpoint['epoch'] + 1
            print(f"Resuming from epoch {start_epoch}")
        else:
            start_epoch = 0
            print("Starting from scratch")
        
        # Training loop
        for epoch in range(epochs):
            model.train()
            epoch_loss = 0
            for images, masks in tqdm(train_loader, desc=f"Epoch {epoch + 1}/{epochs}"):
                # Move to GPU or CPU
                images, masks = images.to(device), masks.to(device)
                
                optimizer.zero_grad()
                
                # Forward pass
                outputs = model(images)
                
                # Compute loss
                loss = bce_loss(outputs, masks)
                loss.backward()
                
                # Update weights
                optimizer.step()
                
                epoch_loss += loss.item()

            print(f"Epoch {epoch + 1}/{epochs}, Loss: {epoch_loss / len(train_loader)}")

            # Validation loop
            model.eval()
            val_loss = 0
            with torch.no_grad():
                for images, masks in val_loader:
                    # Move to GPU or CPU
                    images, masks = images.to(device), masks.to(device)
                    
                    # Forward pass
                    outputs = model(images)
                    
                    # Compute loss
                    loss = bce_loss(outputs, masks)
                    val_loss += loss.item()

            print(f"Validation Loss after Epoch {epoch + 1}: {val_loss / len(val_loader)}")
            
            # Save checkpoint after each epoch
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
            }, checkpoint_path)

        fold += 1

# Set directories for your image and mask data
image_dir = "E:/kits23/png_slices/images/"
mask_dir = "E:/kits23/png_slices/masks/"

# Train the model
train_model(image_dir, mask_dir, epochs=50, batch_size=2, lr=0.001, num_folds=5)


Training fold 1/5...


c:\Users\PRAGNA\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\PRAGNA\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B5_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B5_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Starting from scratch


Epoch 1/50:   0%|          | 87/77564 [03:57<58:38:25,  2.72s/it]


KeyboardInterrupt: 

In [3]:
import os
import shutil
from sklearn.model_selection import train_test_split

def split_dataset(image_dir, mask_dir, output_dir, test_size=0.2):
    # Create output directories
    os.makedirs(output_dir, exist_ok=True)
    for split in ['train', 'test']:
        os.makedirs(os.path.join(output_dir, split, 'images'), exist_ok=True)
        os.makedirs(os.path.join(output_dir, split, 'masks'), exist_ok=True)

    # Get all case folders
    case_folders = os.listdir(image_dir)
    case_folders = [folder for folder in case_folders if os.path.isdir(os.path.join(image_dir, folder))]

    # Split into train and test
    train_cases, test_cases = train_test_split(case_folders, test_size=test_size, random_state=42)

    # Helper function to copy files
    def copy_files(case_list, split):
        for case in case_list:
            # Check if the case is already copied
            dst_images = os.path.join(output_dir, split, 'images', case)
            dst_masks = os.path.join(output_dir, split, 'masks', case)
            
            if os.path.exists(dst_images) and os.path.exists(dst_masks):
                print(f"Skipping {case}, already copied.")
                continue  # Skip this case
            
            # Copy images
            src_images = os.path.join(image_dir, case)
            dst_images = os.path.join(output_dir, split, 'images', case)
            shutil.copytree(src_images, dst_images)

            # Copy masks
            src_masks = os.path.join(mask_dir, case)
            dst_masks = os.path.join(output_dir, split, 'masks', case)
            shutil.copytree(src_masks, dst_masks)

    # Copy data to respective splits
    copy_files(train_cases, 'train')
    copy_files(test_cases, 'test')

    print("Dataset split completed!")
    print(f"Train cases: {len(train_cases)}, Test cases: {len(test_cases)}")

# Usage example
image_dir = "E:/kits23/png_slices/images/"
mask_dir = "E:/kits23/png_slices/masks/"
output_dir = "E:/kits23/split_dataset/"
split_dataset(image_dir, mask_dir, output_dir)


Dataset split completed!
Train cases: 391, Test cases: 98


In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from PIL import Image
import numpy as np
from sklearn.model_selection import KFold
from tqdm import tqdm

# Dataset class to handle image and mask loading
class KidneySegmentationDataset(Dataset):
    def __init__(self, image_dir, mask_dir, transform=None, mask_transform=None):
        self.image_dir = image_dir
        self.mask_dir = mask_dir
        self.transform = transform
        self.mask_transform = mask_transform
        
        # Collect all case folders (e.g., '00000', '00001', ...)
        self.case_folders = os.listdir(image_dir)
        self.case_folders = [folder for folder in self.case_folders if os.path.isdir(os.path.join(image_dir, folder))]
        
        # Calculate total slices across all cases and store slice counts
        self.slice_counts = []
        for case_id in self.case_folders:
            case_image_dir = os.path.join(self.image_dir, case_id)
            # Count number of images in the case
            num_slices = len([f for f in os.listdir(case_image_dir) if f.endswith('.png')])  # assuming images are .png
            self.slice_counts.append(num_slices)
        
    def __len__(self):
        # The length is the total number of slices in all case folders
        return sum(self.slice_counts)
    
    def __getitem__(self, idx):
        # Determine the case folder and slice number for the given index
        cumulative_slices = 0
        for i, num_slices in enumerate(self.slice_counts):
            cumulative_slices += num_slices
            if idx < cumulative_slices:
                case_id = self.case_folders[i]
                slice_id = idx - (cumulative_slices - num_slices)
                break  # Slice indexing
        
        # Build the paths for the image and mask slice
        img_files = sorted([f for f in os.listdir(os.path.join(self.image_dir, case_id)) if f.endswith('.png')])  # Assuming PNG files
        mask_files = sorted([f for f in os.listdir(os.path.join(self.mask_dir, case_id)) if f.endswith('.png')])  # Assuming PNG files
        
        img_path = os.path.join(self.image_dir, case_id, img_files[slice_id])
        mask_path = os.path.join(self.mask_dir, case_id, mask_files[slice_id])

        # Load the image and mask
        img = Image.open(img_path).convert("RGB")  # Convert to RGB (3 channels)
        mask = Image.open(mask_path).convert("L")  # Convert to grayscale (1 channel)
        
        # Apply any transformations if provided
        if self.transform:
            img = self.transform(img)
        if self.mask_transform:
            mask = self.mask_transform(mask)  # The mask transformation will not change the number of channels
        
        return img, mask

# EfficientNet-B5 Encoder (from torchvision)
class EfficientNetB5Encoder(nn.Module):
    def __init__(self, pretrained=True):
        super(EfficientNetB5Encoder, self).__init__()
        # Load the pre-trained EfficientNet-B5 model from torchvision
        self.encoder = models.efficientnet_b5(pretrained=pretrained)
        
        # Remove the classification head (fully connected layers) to retain features
        self.encoder = nn.Sequential(*list(self.encoder.children())[:-1])
    
    def forward(self, x):
        # Extract features from the encoder
        features = self.encoder(x)
        return features

# FPN Decoder
class FPNDecoder(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(FPNDecoder, self).__init__()
        # FPN typically uses several layers to combine features from different scales
        self.conv1 = nn.Conv2d(in_channels, 256, kernel_size=1)
        self.conv2 = nn.Conv2d(256, 128, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(128, 64, kernel_size=3, padding=1)
        self.conv4 = nn.Conv2d(64, out_channels, kernel_size=1)
        
        # Instead of fixed upsampling, use adaptive upsampling
        self.upsample = nn.Upsample(size=(512, 512), mode='bilinear', align_corners=True)  # Adaptive resizing to (512, 512)
    
    def forward(self, x):
        # Decode and combine features from multiple scales
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        
        # Upsample the output to match the input size (512x512)
        x = self.upsample(x)
        return x

# Final model combining EfficientNet-B5 and FPN
class EfficientNetFPN(nn.Module):
    def __init__(self, out_channels=1, pretrained=True):
        super(EfficientNetFPN, self).__init__()
        # Encoder: EfficientNetB5
        self.encoder = EfficientNetB5Encoder(pretrained=pretrained)
        
        # Decoder: FPN
        self.decoder = FPNDecoder(in_channels=2048, out_channels=out_channels)  # EfficientNet-B5 has 2048 channels in the final layer

    def forward(self, x):
        # Forward pass through encoder and decoder
        features = self.encoder(x)
        out = self.decoder(features)
        return out

# Loss Function: Binary Cross-Entropy (BCE)
def bce_loss(output, target):
    return nn.BCEWithLogitsLoss()(output, target)

# Training loop with 5-fold cross-validation
def train_model(image_dir, mask_dir, epochs=50, batch_size=2, lr=0.001, num_folds=5,checkpoint_dir='checkpoints'):
    # Define transformations
    transform = transforms.Compose([  # Transformation for the image
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Example mean/std for RGB images
    ])
    
    mask_transform = transforms.Compose([  # Transformation for the mask
        transforms.ToTensor(),
    ])
    
    # Create checkpoint directory if it doesn't exist
    if not os.path.exists(checkpoint_dir):
        os.makedirs(checkpoint_dir)
    
    # Initialize the dataset
    dataset = KidneySegmentationDataset(image_dir=image_dir, mask_dir=mask_dir, transform=transform, mask_transform=mask_transform)
    
    # Set up cross-validation
    kfold = KFold(n_splits=num_folds, shuffle=True)
    
    fold = 1
    for train_idx, val_idx in kfold.split(dataset):
        print(f"Training fold {fold}/{num_folds}...")
        
        # Split the dataset into training and validation sets for this fold
        train_subset = torch.utils.data.Subset(dataset, train_idx)
        val_subset = torch.utils.data.Subset(dataset, val_idx)
        
        # Data loaders
        train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=True)
        val_loader = DataLoader(val_subset, batch_size=batch_size, shuffle=False)
        
        # Initialize model, optimizer, and loss function
        model = EfficientNetFPN(out_channels=1, pretrained=True)
        
        # Check if CUDA is available, else use CPU
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        model = model.to(device)
        
        optimizer = optim.Adam(model.parameters(), lr=lr)
        
        # Load checkpoint if it exists
        checkpoint_path = os.path.join(checkpoint_dir, f"fold_{fold}_checkpoint.pth")
        if os.path.exists(checkpoint_path):
            checkpoint = torch.load(checkpoint_path)
            model.load_state_dict(checkpoint['model_state_dict'])
            optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
            start_epoch = checkpoint['epoch'] + 1
            print(f"Resuming from epoch {start_epoch}")
        else:
            start_epoch = 0
            print("Starting from scratch")
        
        # Training loop
        for epoch in range(epochs):
            model.train()
            epoch_loss = 0
            for images, masks in tqdm(train_loader, desc=f"Epoch {epoch + 1}/{epochs}"):
                # Move to GPU or CPU
                images, masks = images.to(device), masks.to(device)
                
                optimizer.zero_grad()
                
                # Forward pass
                outputs = model(images)
                
                # Compute loss
                loss = bce_loss(outputs, masks)
                loss.backward()
                
                # Update weights
                optimizer.step()
                
                epoch_loss += loss.item()

            print(f"Epoch {epoch + 1}/{epochs}, Loss: {epoch_loss / len(train_loader)}")

            # Validation loop
            model.eval()
            val_loss = 0
            with torch.no_grad():
                for images, masks in val_loader:
                    # Move to GPU or CPU
                    images, masks = images.to(device), masks.to(device)
                    
                    # Forward pass
                    outputs = model(images)
                    
                    # Compute loss
                    loss = bce_loss(outputs, masks)
                    val_loss += loss.item()

            print(f"Validation Loss after Epoch {epoch + 1}: {val_loss / len(val_loader)}")
            
            # Save checkpoint after each epoch
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
            }, checkpoint_path)

        fold += 1

# Set directories for your image and mask data
image_dir = "E:/kits23/split_dataset/train/images/"
mask_dir = "E:/kits23/split_dataset/train/masks/"

# Train the model
train_model(image_dir, mask_dir, epochs=50, batch_size=2, lr=0.001, num_folds=5)


Training fold 1/5...


c:\Users\PRAGNA\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\PRAGNA\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B5_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B5_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Starting from scratch


Epoch 1/50:   0%|          | 22/62088 [01:12<56:29:46,  3.28s/it]


KeyboardInterrupt: 

In [2]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from PIL import Image
import numpy as np
from sklearn.model_selection import KFold
from tqdm import tqdm

# Dataset class to handle image and mask loading
class KidneySegmentationDataset(Dataset):
    def __init__(self, image_dir, mask_dir, transform=None, mask_transform=None):
        self.image_dir = image_dir
        self.mask_dir = mask_dir
        self.transform = transform
        self.mask_transform = mask_transform
        
        # Collect all case folders (e.g., '00000', '00001', ...)
        self.case_folders = os.listdir(image_dir)
        self.case_folders = [folder for folder in self.case_folders if os.path.isdir(os.path.join(image_dir, folder))]
        
        # Calculate total slices across all cases and store slice counts
        self.slice_counts = []
        for case_id in self.case_folders:
            case_image_dir = os.path.join(self.image_dir, case_id)
            # Count number of images in the case
            num_slices = len([f for f in os.listdir(case_image_dir) if f.endswith('.png')])  # assuming images are .png
            self.slice_counts.append(num_slices)
        
    def __len__(self):
        # The length is the total number of slices in all case folders
        return sum(self.slice_counts)
    
    def __getitem__(self, idx):
        # Determine the case folder and slice number for the given index
        cumulative_slices = 0
        for i, num_slices in enumerate(self.slice_counts):
            cumulative_slices += num_slices
            if idx < cumulative_slices:
                case_id = self.case_folders[i]
                slice_id = idx - (cumulative_slices - num_slices)
                break  # Slice indexing
        
        # Build the paths for the image and mask slice
        img_files = sorted([f for f in os.listdir(os.path.join(self.image_dir, case_id)) if f.endswith('.png')])  # Assuming PNG files
        mask_files = sorted([f for f in os.listdir(os.path.join(self.mask_dir, case_id)) if f.endswith('.png')])  # Assuming PNG files
        
        img_path = os.path.join(self.image_dir, case_id, img_files[slice_id])
        mask_path = os.path.join(self.mask_dir, case_id, mask_files[slice_id])

        # Load the image and mask
        img = Image.open(img_path).convert("RGB")  # Convert to RGB (3 channels)
        mask = Image.open(mask_path).convert("L")  # Convert to grayscale (1 channel)
        
        # Apply any transformations if provided
        if self.transform:
            img = self.transform(img)
        if self.mask_transform:
            mask = self.mask_transform(mask)  # The mask transformation will not change the number of channels
        
        return img, mask

# EfficientNet-B5 Encoder (from torchvision)
class EfficientNetB5Encoder(nn.Module):
    def __init__(self, pretrained=True):
        super(EfficientNetB5Encoder, self).__init__()
        # Load the pre-trained EfficientNet-B5 model from torchvision
        self.encoder = models.efficientnet_b5(pretrained=pretrained)
        
        # Remove the classification head (fully connected layers) to retain features
        self.encoder = nn.Sequential(*list(self.encoder.children())[:-1])
    
    def forward(self, x):
        # Extract features from the encoder
        features = self.encoder(x)
        return features

# FPN Decoder
class FPNDecoder(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(FPNDecoder, self).__init__()
        # FPN typically uses several layers to combine features from different scales
        self.conv1 = nn.Conv2d(in_channels, 256, kernel_size=1)
        self.conv2 = nn.Conv2d(256, 128, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(128, 64, kernel_size=3, padding=1)
        self.conv4 = nn.Conv2d(64, out_channels, kernel_size=1)
        
        # Instead of fixed upsampling, use adaptive upsampling
        self.upsample = nn.Upsample(size=(512, 512), mode='bilinear', align_corners=True)  # Adaptive resizing to (512, 512)
    
    def forward(self, x):
        # Decode and combine features from multiple scales
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        
        # Upsample the output to match the input size (512x512)
        x = self.upsample(x)
        return x

# Final model combining EfficientNet-B5 and FPN
class EfficientNetFPN(nn.Module):
    def __init__(self, out_channels=1, pretrained=True):
        super(EfficientNetFPN, self).__init__()
        # Encoder: EfficientNetB5
        self.encoder = EfficientNetB5Encoder(pretrained=pretrained)
        
        # Decoder: FPN
        self.decoder = FPNDecoder(in_channels=2048, out_channels=out_channels)  # EfficientNet-B5 has 2048 channels in the final layer

    def forward(self, x):
        # Forward pass through encoder and decoder
        features = self.encoder(x)
        out = self.decoder(features)
        return out

# Loss Function: Binary Cross-Entropy (BCE)
def bce_loss(output, target):
    return nn.BCEWithLogitsLoss()(output, target)

# Training loop with 5-fold cross-validation
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, Subset
from tqdm import tqdm

def train_model(image_dir, mask_dir, epochs=50, batch_size=2, lr=0.001, num_folds=5, checkpoint_dir='checkpoints'):
    # Define transformations
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    mask_transform = transforms.Compose([
        transforms.ToTensor(),
    ])

    # Create checkpoint directory if it doesn't exist
    os.makedirs(checkpoint_dir, exist_ok=True)

    # Initialize the dataset
    dataset = KidneySegmentationDataset(image_dir=image_dir, mask_dir=mask_dir, transform=transform, mask_transform=mask_transform)

    # Set up cross-validation
    kfold = KFold(n_splits=num_folds, shuffle=True)

    for fold, (train_idx, val_idx) in enumerate(kfold.split(dataset), start=1):
        print(f"Training fold {fold}/{num_folds}...")

        # Split dataset
        train_subset = Subset(dataset, train_idx)
        val_subset = Subset(dataset, val_idx)

        # Data loaders
        train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=True)
        val_loader = DataLoader(val_subset, batch_size=batch_size, shuffle=False)

        # Initialize model, optimizer, and loss function
        model = EfficientNetFPN(out_channels=1, pretrained=True)
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        model = model.to(device)
        optimizer = optim.Adam(model.parameters(), lr=lr)

        # Load checkpoint if exists
        checkpoint_path = os.path.join(checkpoint_dir, f"fold_{fold}_checkpoint.pth")
        if os.path.exists(checkpoint_path):
            checkpoint = torch.load(checkpoint_path)
            model.load_state_dict(checkpoint['model_state_dict'])
            optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
            start_epoch = checkpoint['epoch']
            start_batch = checkpoint['batch']
            print(f"Resuming from epoch {start_epoch + 1}, batch {start_batch + 1}")
        else:
            start_epoch = 0
            start_batch = 0
            print("Starting from scratch")
            
            

        # Training loop
        for epoch in range(start_epoch, epochs):
            model.train()
            epoch_loss = 0

            for batch_idx, (images, masks) in enumerate(tqdm(train_loader, desc=f"Epoch {epoch + 1}/{epochs}"), start=start_batch):
                # Skip already processed batches
                if batch_idx < start_batch:
                    continue

                # Move to device
                images, masks = images.to(device), masks.to(device)

                optimizer.zero_grad()

                # Forward pass
                outputs = model(images)
                loss = bce_loss(outputs, masks)
                loss.backward()
                optimizer.step()

                epoch_loss += loss.item()

                # Save progress after each batch
                torch.save({
                    'epoch': epoch,
                    'batch': batch_idx,
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                }, checkpoint_path)

            print(f"Epoch {epoch + 1}/{epochs}, Loss: {epoch_loss / len(train_loader)}")

            # Validation loop
            model.eval()
            val_loss = 0
            with torch.no_grad():
                for images, masks in val_loader:
                    images, masks = images.to(device), masks.to(device)
                    outputs = model(images)
                    loss = bce_loss(outputs, masks)
                    val_loss += loss.item()

            print(f"Validation Loss after Epoch {epoch + 1}: {val_loss / len(val_loader)}")

            # Reset start_batch for the next epoch
            start_batch = 0


# Set directories for your image and mask data
image_dir = "E:/kits23/split_dataset/train/images/"
mask_dir = "E:/kits23/split_dataset/train/masks/"

# Train the model
train_model(image_dir, mask_dir, epochs=50, batch_size=2, lr=0.001, num_folds=5)


Training fold 1/5...


C:\Users\PRAGNA\AppData\Local\Temp\ipykernel_14992\168382821.py:171: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint_path)


KeyError: "filename 'storages' not found"

In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, Subset
from torchvision import transforms, models
from PIL import Image
from sklearn.model_selection import KFold
from tqdm import tqdm
import tempfile

# Dataset class
class KidneySegmentationDataset(Dataset):
    def __init__(self, image_dir, mask_dir, transform=None, mask_transform=None):
        self.image_dir = image_dir
        self.mask_dir = mask_dir
        self.transform = transform
        self.mask_transform = mask_transform
        self.case_folders = [
            folder for folder in os.listdir(image_dir) if os.path.isdir(os.path.join(image_dir, folder))
        ]
        self.slice_counts = [
            len([f for f in os.listdir(os.path.join(image_dir, case)) if f.endswith('.png')])
            for case in self.case_folders
        ]

    def __len__(self):
        return sum(self.slice_counts)

    def __getitem__(self, idx):
        cumulative_slices = 0
        for i, num_slices in enumerate(self.slice_counts):
            cumulative_slices += num_slices
            if idx < cumulative_slices:
                case_id = self.case_folders[i]
                slice_id = idx - (cumulative_slices - num_slices)
                break
        
        img_files = sorted([f for f in os.listdir(os.path.join(self.image_dir, case_id)) if f.endswith('.png')])
        mask_files = sorted([f for f in os.listdir(os.path.join(self.mask_dir, case_id)) if f.endswith('.png')])
        img_path = os.path.join(self.image_dir, case_id, img_files[slice_id])
        mask_path = os.path.join(self.mask_dir, case_id, mask_files[slice_id])
        
        img = Image.open(img_path).convert("RGB")
        mask = Image.open(mask_path).convert("L")
        if self.transform:
            img = self.transform(img)
        if self.mask_transform:
            mask = self.mask_transform(mask)
        return img, mask

# Model components
class EfficientNetB5Encoder(nn.Module):
    def __init__(self, pretrained=True):
        super(EfficientNetB5Encoder, self).__init__()
        self.encoder = models.efficientnet_b5(weights="DEFAULT" if pretrained else None)
        self.encoder = nn.Sequential(*list(self.encoder.children())[:-1])

    def forward(self, x):
        return self.encoder(x)

class FPNDecoder(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(FPNDecoder, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, 256, kernel_size=1)
        self.conv2 = nn.Conv2d(256, 128, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(128, 64, kernel_size=3, padding=1)
        self.conv4 = nn.Conv2d(64, out_channels, kernel_size=1)
        self.upsample = nn.Upsample(size=(512, 512), mode='bilinear', align_corners=True)

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        return self.upsample(x)

class EfficientNetFPN(nn.Module):
    def __init__(self, out_channels=1, pretrained=True):
        super(EfficientNetFPN, self).__init__()
        self.encoder = EfficientNetB5Encoder(pretrained=pretrained)
        self.decoder = FPNDecoder(in_channels=2048, out_channels=out_channels)

    def forward(self, x):
        features = self.encoder(x)
        return self.decoder(features)

# Loss function
def bce_loss(output, target):
    return nn.BCEWithLogitsLoss()(output, target)

# Atomic save function
def atomic_save(data, file_path):
    temp_path = tempfile.NamedTemporaryFile(delete=False, dir=os.path.dirname(file_path)).name
    torch.save(data, temp_path)
    os.replace(temp_path, file_path)

# Training loop
def train_model(image_dir, mask_dir, epochs=50, batch_size=2, lr=0.001, num_folds=5, checkpoint_dir='checkpoints'):
    transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])
    mask_transform = transforms.ToTensor()
    os.makedirs(checkpoint_dir, exist_ok=True)
    dataset = KidneySegmentationDataset(image_dir=image_dir, mask_dir=mask_dir, transform=transform, mask_transform=mask_transform)
    kfold = KFold(n_splits=num_folds, shuffle=True)

    for fold, (train_idx, val_idx) in enumerate(kfold.split(dataset), start=1):
        print(f"Training fold {fold}/{num_folds}...")
        train_loader = DataLoader(Subset(dataset, train_idx), batch_size=batch_size, shuffle=True)
        val_loader = DataLoader(Subset(dataset, val_idx), batch_size=batch_size, shuffle=False)
        model = EfficientNetFPN(out_channels=1, pretrained=True).to("cuda" if torch.cuda.is_available() else "cpu")
        optimizer = optim.Adam(model.parameters(), lr=lr)
        checkpoint_path = os.path.join(checkpoint_dir, f"fold_{fold}_checkpoint.pth")

        start_epoch, start_batch = 0, 0
        if os.path.exists(checkpoint_path):
            try:
                checkpoint = torch.load(checkpoint_path)
                model.load_state_dict(checkpoint['model_state_dict'])
                optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
                start_epoch = checkpoint['epoch']
                start_batch = checkpoint['batch']
                print(f"Resuming from epoch {start_epoch + 1}, batch {start_batch + 1}")
            except Exception as e:
                print(f"Failed to load checkpoint for fold {fold}: {e}")
                print("Starting from scratch for this fold.")

        for epoch in range(start_epoch, epochs):
            model.train()
            epoch_loss = 0
            for batch_idx, (images, masks) in enumerate(tqdm(train_loader, desc=f"Epoch {epoch + 1}/{epochs}"), start=start_batch):
                images, masks = images.to("cuda"), masks.to("cuda")
                optimizer.zero_grad()
                outputs = model(images)
                loss = bce_loss(outputs, masks)
                loss.backward()
                optimizer.step()
                epoch_loss += loss.item()
                atomic_save({'epoch': epoch, 'batch': batch_idx, 'model_state_dict': model.state_dict(), 'optimizer_state_dict': optimizer.state_dict()}, checkpoint_path)
            print(f"Epoch {epoch + 1}/{epochs}, Loss: {epoch_loss / len(train_loader)}")
            val_loss = 0
            model.eval()
            with torch.no_grad():
                for images, masks in val_loader:
                    images, masks = images.to("cuda"), masks.to("cuda")
                    val_loss += bce_loss(model(images), masks).item()
            print(f"Validation Loss after Epoch {epoch + 1}: {val_loss / len(val_loader)}")
            start_batch = 0

# Set directories for image and mask data
image_dir = "E:/kits23/split_dataset/train/images/"
mask_dir = "E:/kits23/split_dataset/train/masks/"
train_model(image_dir, mask_dir, epochs=50, batch_size=2, lr=0.001, num_folds=5)


In [22]:
import os
import numpy as np
from sklearn.model_selection import KFold
import nibabel as nib
import torch
from torch.utils.data import Dataset, DataLoader, Subset

class SliceLevelDataset(Dataset):
    def __init__(self, image_dir, mask_dir, slice_metadata, transform=None):
        self.image_dir = image_dir
        self.mask_dir = mask_dir
        self.slice_metadata = slice_metadata  # List of tuples (case_folder, slice_idx)
        self.transform = transform

    def __len__(self):
        return len(self.slice_metadata)

    def __getitem__(self, idx):
        case_folder, slice_idx = self.slice_metadata[idx]
        
        # Load the specific slice
        image_path = os.path.join(self.image_dir, case_folder, f"slice_{slice_idx}.nii.gz")
        mask_path = os.path.join(self.mask_dir, case_folder, f"slice_{slice_idx}.nii.gz")
        
        image = nib.load(image_path).get_fdata()
        mask = nib.load(mask_path).get_fdata()
        
        # Convert to torch tensors
        image_tensor = torch.tensor(image, dtype=torch.float32).unsqueeze(0)  # Add channel dimension
        mask_tensor = torch.tensor(mask, dtype=torch.float32).unsqueeze(0)
        
        if self.transform:
            image_tensor = self.transform(image_tensor)
            mask_tensor = self.transform(mask_tensor)
        
        return image_tensor, mask_tensor


def create_slice_metadata(image_dir):
    """
    Create a metadata list where each entry corresponds to (case_folder, slice_idx).
    """
    metadata = []
    case_folders = sorted(os.listdir(image_dir))
    
    for case_folder in case_folders:
        slice_files = sorted(os.listdir(os.path.join(image_dir, case_folder)))
        for slice_file in slice_files:
            slice_idx = slice_file.split('_')[1].replace('.nii.gz', '')  # Assuming naming like 'slice_000.nii.gz'
            metadata.append((case_folder, slice_idx))
    
    return metadata


def split_slices_kfold(slice_metadata, k=5, random_state=42):
    """
    Split the slices into K-Folds while ensuring case-level integrity.
    """
    # Extract unique case identifiers
    unique_cases = sorted({case for case, _ in slice_metadata})
    case_to_slices = {case: [] for case in unique_cases}
    
    for case, slice_idx in slice_metadata:
        case_to_slices[case].append(slice_idx)
    
    # K-Fold splitting based on cases
    kf = KFold(n_splits=k, shuffle=True, random_state=random_state)
    case_indices = np.arange(len(unique_cases))
    
    folds = []
    for train_indices, val_indices in kf.split(case_indices):
        train_cases = [unique_cases[i] for i in train_indices]
        val_cases = [unique_cases[i] for i in val_indices]
        
        train_slices = [(case, idx) for case in train_cases for idx in case_to_slices[case]]
        val_slices = [(case, idx) for case in val_cases for idx in case_to_slices[case]]
        
        folds.append((train_slices, val_slices))
    
    return folds


# Example usage
image_dir = 'E:/kits23/2d_slices/images/'
mask_dir = 'E:/kits23/2d_slices/masks/'

# Step 1: Create slice-level metadata
slice_metadata = create_slice_metadata(image_dir)

# Step 2: Perform K-Fold splitting
folds = split_slices_kfold(slice_metadata, k=5)

# Step 3: Create datasets and dataloaders for a specific fold
fold_idx = 0  # For the first fold
train_slices, val_slices = folds[fold_idx]

train_dataset = SliceLevelDataset(image_dir, mask_dir, train_slices)
val_dataset = SliceLevelDataset(image_dir, mask_dir, val_slices)

train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=2, shuffle=False)

# Now you can use train_loader and val_loader for training!
